In [1]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shaper import BookShaper
import numpy as np
from pylab import rcParams

In [2]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [3]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [4]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [8]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed
from deep_orderbook.replayer import ParquetReplayer

async def iter_sec(date_regexp='2024-08-06', max_samples=0):
    replayer = ParquetReplayer('data', date_regexp=date_regexp)
    async with CoinbaseFeed(
        markets=MARKETS,
        replayer=replayer,
    ) as feed:
        async for onesec in feed.one_second_iterator():
            yield onesec
            if max_samples == 0:
                break
            else:
                max_samples -= 1

async for onesec in iter_sec(max_samples=2):
    print(onesec)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

One second: 2024-08-06 08:25:44+00:00 
ETH-BTC: BBO: price=0.04485 size=0.42857832-price=0.04486 size=7.84479889, num trades: 0
BTC-USD: BBO: price=55670.64 size=0.015-price=55673.3 size=0.05, num trades: 0
ETH-USD: BBO: price=2496.57 size=8.01044562-price=2496.98 size=1.52612798, num trades: 2

One second: 2024-08-06 08:25:45+00:00 
ETH-BTC: BBO: price=0.04485 size=0.008-price=0.04486 size=7.84479889, num trades: 0
BTC-USD: BBO: price=55670.8 size=0.015-price=55673.16 size=0.00034315, num trades: 0
ETH-USD: BBO: price=2496.69 size=0.7875-price=2496.83 size=0.0134929, num trades: 2

One second: 2024-08-06 08:25:46+00:00 
ETH-BTC: BBO: price=0.04485 size=1.25868949-price=0.04486 size=6.44266125, num trades: 0
BTC-USD: BBO: price=55676.92 size=0.0731-price=55678.71 size=0.10779736, num trades: 6
ETH-USD: BBO: price=2497.16 size=2.80317839-price=2497.2 size=0.36278888, num trades: 1



In [10]:
async def iter_shapes(date_regexp='2024-08-06', max_samples=0):
    symbol_shapers = {pair: BookShaper() for pair in MARKETS}
    async for onesec in iter_sec(max_samples=2):
        yield {
            market: await symbol_shapers[market].update(
                onesec.symbols[market], onesec.ts
            )
            for market in MARKETS
        }


async for shaped in iter_shapes():
    print(shaped)


  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

{'ETH-BTC': {'time': datetime.datetime(2024, 8, 6, 8, 25, 44, tzinfo=TzInfo(UTC)), 'price': 0.044850520001, 'bids':                                    size
price                                  
(price, 0.04485)     (size, 0.42857832)
(price, 0.04484)    (size, 13.87024489)
(price, 0.04483)    (size, 28.55443584)
(price, 0.04482)     (size, 24.0507326)
(price, 0.04481)    (size, 10.56853926)
...                                 ...
(price, 5e-05)            (size, 1.988)
(price, 4e-05)            (size, 2.485)
(price, 3e-05)    (size, 2503.31377333)
(price, 2e-05)             (size, 4.97)
(price, 1e-05)        (size, 153.97466)

[816 rows x 1 columns], 'asks':                                  size
price                                
(price, 0.04486)   (size, 7.84479889)
(price, 0.04487)  (size, 19.49929362)
(price, 0.04488)  (size, 28.24915108)
(price, 0.04489)  (size, 13.78963234)
(price, 0.0449)   (size, 29.88812237)
...                               ...
(price, 40.0)           (si

In [ ]:
multi_replay = file_replayer.multireplayL2_async(MARKETS)
d = await next(multi_replay)
d

In [ ]:
async def run(maxpoints=10):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())
await task

In [13]:
async def iter_gen_arrays(date_regexp='2024-08-06', max_samples=0):
    shaper = BookShaper()
    async for onesec in iter_sec(max_samples=max_samples):
        yield await shaper.gen_array_async(onesec)

async for arr in iter_gen_arrays(max_samples=2):
    print(arr)

genarr = shaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)
_

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

TypeError: BookShaper.gen_array_async() missing 1 required positional argument: 'markets'

In [ ]:
genacc = shaper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)
_

In [ ]:
async for toshow in BookShaper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
    plt.figure(figsize=(15,11))
    plt.imshow(toshow, origin="upper")
    clear_output(wait=True)
    plt.show()

In [ ]:
every = 2
LENGTH = 256
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower")
        plt.show()
        # pr = np.stack(data['ps'])[-LENGTH:, :-1]
        # plt.plot(pr)
        # plt.show()
#        print(symb)
        clear_output(wait=True)
#        plt.show()
#    if n == 100:
#        break
    pass